In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install symspellpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 9.5 MB/s eta 0:00:00


In [5]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
import nltk
nltk.download('punkt', download_dir='/path/to/nltk_data')


[nltk_data] Downloading package punkt to /path/to/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
import nltk
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [9]:
from nltk.tokenize import word_tokenize

text = "This is a test sentence."
tokens = word_tokenize(text)
print(tokens)


['This', 'is', 'a', 'test', 'sentence', '.']


In [10]:
from symspellpy import SymSpell, Verbosity
from nltk import bigrams, FreqDist
from nltk.tokenize import word_tokenize
import nltk
import sys
import os

# Ensure terminal uses UTF-8 encoding
if os.name == 'nt':
    sys.stdout.reconfigure(encoding='utf-8')

# Ensure NLTK punkt tokenizer is available
try:
    nltk.data.find("tokenizers/punkt")
except LookupError:
    print("Downloading 'punkt' tokenizer...")
    nltk.download("punkt")

# Initialize the spell checker
def initialize_spell_checker(dictionary_path):
    """
    Initialize SymSpell with the provided dictionary file.
    Args:
        dictionary_path (str): Path to the dictionary file.
    Returns:
        SymSpell: Initialized SymSpell object.
    """
    sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
    try:
        with open(dictionary_path, "r", encoding="utf-8") as file:
            for line in file:
                parts = line.strip().split()
                if len(parts) == 2 and parts[1].isdigit():
                    sym_spell.create_dictionary_entry(parts[0], int(parts[1]))
    except FileNotFoundError:
        raise FileNotFoundError(f"Dictionary file not found: {dictionary_path}")
    return sym_spell

# Build the bigram model using NLTK
def generate_bigrams(tokens):
    """
    Generate bigrams from a list of tokens using NLTK.
    Args:
        tokens (list): List of tokens (words).
    Returns:
        FreqDist: A frequency distribution of bigrams.
    """
    bigram_list = list(bigrams(tokens))
    return FreqDist(bigram_list)

# Calculate Levenshtein distance
def levenshtein_distance(word1, word2):
    """
    Calculate the Levenshtein distance between two words.
    Args:
        word1 (str): First word.
        word2 (str): Second word.
    Returns:
        int: The Levenshtein distance.
    """
    m, n = len(word1), len(word2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif word1[i - 1] == word2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])

    return dp[m][n]

# Perform combined spell checking
def spell_check_combined(text, sym_spell, bigram_model):
    """
    Perform spell checking by integrating a bigram model and Levenshtein distance.
    Args:
        text (str): Input text to spell check.
        sym_spell (SymSpell): Initialized SymSpell object.
        bigram_model (FreqDist): Bigram language model built from a corpus.
    Returns:
        tuple: Corrected text, list of corrections made, and dictionary of suggestions.
    """
    if not text.strip():
        return "", [], {}

    words = word_tokenize(text.lower())
    corrected_text = []
    corrections = []
    suggestions_dict = {}

    for i, word in enumerate(words):
        suggestions = sym_spell.lookup(word, Verbosity.ALL, max_edit_distance=2)
        if suggestions:
            ranked_suggestions = sorted(
                suggestions, key=lambda s: levenshtein_distance(word, s.term)
            )
            corrected_word = ranked_suggestions[0].term
            corrected_text.append(corrected_word)

            if corrected_word != word:
                corrections.append((word, corrected_word))
                suggestions_dict[word] = [s.term for s in ranked_suggestions[:3]]
        else:
            corrected_text.append(word)

    return " ".join(corrected_text), corrections, suggestions_dict

# Main Function to Test Spell Checking
def main():
    try:
        dictionary_path = r"/content/drive/MyDrive/tamilWords.txt"
        sym_spell = initialize_spell_checker(dictionary_path)

        corpus = """
        அம்மா மரம் பக்கத்திற்கு சென்றார்.
        அவள் பூசையைக் கவனிக்கவில்லை.
        குழந்தைகள் பள்ளியில் விளையாடினர்.
        """
        tokens = word_tokenize(corpus.lower())
        bigram_model = generate_bigrams(tokens)

        # Add test sentences with Tamil spelling mistakes
        test_sentences = [
            "வீட்டுக்க கடிகம அனப்பா வென்டும",
            "படசாலை சொல்லா வோண்டூம்",
            "அம்மூ பாழம் வாங்கீ வந்தூர்",
            "நானீ கெழும்பூ பேக வோண்டும்",
            "அருமை நண்பருக்கு மன்மதர ராசிபலம்"
        ]

        for sentence in test_sentences:
            print(f"\nOriginal Sentence: {sentence}")
            corrected_text, corrections, suggestions_dict = spell_check_combined(sentence, sym_spell, bigram_model)
            print(f"Corrected Text: {corrected_text}")
            if corrections:
                print("Corrections Made:")
                for original, corrected in corrections:
                    suggestions = ", ".join(suggestions_dict.get(original, []))
                    print(f"  {original} -> {corrected} (Suggestions: {suggestions})")
            else:
                print("No corrections made.")

    except Exception as e:
        print(f"Failed to initialize resources: {e}")

if __name__ == "__main__":
    main()


Original Sentence: வீட்டுக்க கடிகம அனப்பா வென்டும
Corrected Text: விட்டுக் கடிகை அப்பா வென்று
Corrections Made:
  வீட்டுக்க -> விட்டுக் (Suggestions: விட்டுக்)
  கடிகம -> கடிகை (Suggestions: கடிகை, நடிகை, கரிம)
  அனப்பா -> அப்பா (Suggestions: அப்பா, அரப்பா, அன்பு)
  வென்டும -> வென்று (Suggestions: வென்று, வெட்டும், வெட்டு)

Original Sentence: படசாலை சொல்லா வோண்டூம்
Corrected Text: பாடசாலை சொல்லா வேண்டும்
Corrections Made:
  படசாலை -> பாடசாலை (Suggestions: பாடசாலை, பேசாலை, சாலை)
  வோண்டூம் -> வேண்டும் (Suggestions: வேண்டும், வேண்டாம், கோண்டூர்)

Original Sentence: அம்மூ பாழம் வாங்கீ வந்தூர்
Corrected Text: அம்மா பாலம் வாழ்க வந்தார்
Corrections Made:
  அம்மூ -> அம்மா (Suggestions: அம்மா, அம்மை, அம்ம)
  பாழம் -> பாலம் (Suggestions: பாலம், பழம், பாகம்)
  வாங்கீ -> வாழ்க (Suggestions: வாழ்க, வாக்கு, வாழ்கை)
  வந்தூர் -> வந்தார் (Suggestions: வந்தார், வந்தனர், வந்தவர்)

Original Sentence: நானீ கெழும்பூ பேக வோண்டும்
Corrected Text: நான் கொழும்பு பே வேண்டும்
Corrections Made:
  நானீ -> நான் (